<a href="https://colab.research.google.com/github/ludoveltz/test_github_fev25/blob/main/Exc_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q rouge-score transformers torch tqdm


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.4 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

# Affichons le contenu pour trouver le bon chemin
!ls "/content/drive/MyDrive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Colab Notebooks'  'Dogs vs Cats'		      metadata.csv
 Data		    household_power_consumption.txt   reponses_emotionnelles.json
 DATASET	   'IMDB Dataset.csv'		      structure_dossiers.json


In [10]:
import pandas as pd
import numpy as np
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    GPT2LMHeadModel,
    GPT2Tokenizer
)
from rouge_score import rouge_scorer
from typing import Dict, List
from tqdm import tqdm
import torch
from google.colab import drive

class ModelComparator:
    def __init__(self, train_df: pd.DataFrame, test_df: pd.DataFrame):
            """
            Initialisation des modèles et des tokenizers avec optimisation GPU
            """
            self.train_df = train_df
            self.test_df = test_df
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

            if self.device.type == "cuda":
                torch.cuda.empty_cache()  # Nettoyage de la mémoire GPU

            print(f"🖥️ Utilisation de: {self.device}")

            # Initialisation des modèles
            self.models = {
                't5-small': (
                    T5ForConditionalGeneration.from_pretrained('t5-small').to(self.device),
                    T5Tokenizer.from_pretrained('t5-small')
                ),
                't5-base': (
                    T5ForConditionalGeneration.from_pretrained('t5-base').to(self.device),
                    T5Tokenizer.from_pretrained('t5-base')
                ),
                'gpt2': (
                    GPT2LMHeadModel.from_pretrained('gpt2').to(self.device),
                    GPT2Tokenizer.from_pretrained('gpt2')
                )
            }

            self.rouge_scorer = rouge_scorer.RougeScorer(
                ['rouge1', 'rouge2', 'rougeL'],
                use_stemmer=True
            )


    def generate_summary(self, text: str, model_name: str) -> str:
        """
        Génération de résumé unifiée pour tous les modèles
        """
        try:
            if model_name.startswith('t5'):
                return self._generate_t5_summary(text, model_name)
            else:
                return self._generate_gpt2_summary(text)
        except Exception as e:
            print(f"⚠️ Erreur génération résumé ({model_name}): {e}")
            return ""

    def _generate_t5_summary(self, text: str, model_name: str) -> str:
        model, tokenizer = self.models[model_name]
        input_text = f"summarize: {text}"
        inputs = tokenizer(input_text, max_length=512, truncation=True, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                max_length=150,
                min_length=40,
                num_beams=4,
                length_penalty=2.0
            )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    def _generate_gpt2_summary(self, text: str) -> str:
        """
        Génération de résumé avec GPT-2 avec gestion correcte du padding
        """
        model, tokenizer = self.models['gpt2']

        # Configuration du pad_token
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = model.config.eos_token_id

        input_text = f"{text}\nTL;DR:"
        inputs = tokenizer(
            input_text,
            max_length=1024,
            truncation=True,
            padding=True,
            return_tensors="pt"
        ).to(self.device)

        try:
            with torch.no_grad():
                outputs = model.generate(
                    inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=150,
                    min_length=30,
                    num_beams=4,
                    length_penalty=2.0,
                    pad_token_id=tokenizer.pad_token_id
                )

            summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
            return summary.split("TL;DR:")[-1].strip()

        except Exception as e:
            print(f"⚠️ Erreur génération GPT-2: {str(e)}")
            return ""



    def compare_models(self, sample_size: int = 50) -> pd.DataFrame:
        # Modification de l'échantillonnage stratifié
        sample_df = self.train_df.groupby('language', group_keys=False).apply(
            lambda x: x.sample(min(len(x), sample_size//len(self.train_df['language'].unique())))
        ).reset_index(drop=True)

        results = []

        for idx, row in tqdm(sample_df.iterrows(), desc="Comparaison des modèles"):
            for model_name in self.models.keys():
                prediction = self.generate_summary(row['premise'], model_name)
                scores = self.rouge_scorer.score(row['hypothesis'], prediction)

                results.append({
                    'model': model_name,
                    'language': row['language'],
                    'rouge1': scores['rouge1'].fmeasure,
                    'rouge2': scores['rouge2'].fmeasure,
                    'rougeL': scores['rougeL'].fmeasure
                })

        return pd.DataFrame(results)

    def compare_models_summaries(self, sample_size: int = 5) -> pd.DataFrame:
        """
        Comparaison côte à côte des résumés générés
        """
        # Échantillonnage stratifié plus petit pour la lisibilité
        sample_df = self.train_df.groupby('language').apply(
            lambda x: x.sample(min(len(x), sample_size))
        ).reset_index(drop=True)

        results = []

        for idx, row in tqdm(sample_df.iterrows(), desc="Génération des résumés"):
            summary_row = {
                'original_text': row['premise'],
                'reference': row['hypothesis'],
                'language': row['language']
            }

            # Génération des résumés pour chaque modèle
            for model_name in self.models.keys():
                summary = self.generate_summary(row['premise'], model_name)
                summary_row[f'{model_name}_summary'] = summary

                # Calcul des scores ROUGE
                scores = self.rouge_scorer.score(row['hypothesis'], summary)
                for metric in ['rouge1', 'rouge2', 'rougeL']:
                    summary_row[f'{model_name}_{metric}'] = scores[metric].fmeasure

            results.append(summary_row)

        return pd.DataFrame(results)

def display_comparison_results(aggregated_scores: pd.DataFrame, summary_comparison: pd.DataFrame):
    print("\n📊 Scores ROUGE agrégés par modèle:")

    # Formatage des scores moyens
    mean_scores = aggregated_scores.groupby('model')[['rouge1', 'rouge2', 'rougeL']].mean()
    print("\nMoyenne globale:")
    print(mean_scores.round(4))

    # Formatage des scores par langue
    lang_scores = aggregated_scores.groupby(['language', 'model'])[['rouge1', 'rouge2', 'rougeL']].mean()
    print("\nMoyenne par langue et modèle:")
    print(lang_scores.round(4))

    # Ajout des écarts-types
    print("\nÉcart-type par modèle:")
    print(aggregated_scores.groupby('model')[['rouge1', 'rouge2', 'rougeL']].std().round(4))


def main():
    try:
        # Montage du Drive
        drive.mount('/content/drive')

        # Chemin vers les données
        base_path = "/content/drive/MyDrive/DATASET"

        # Chargement des données train uniquement
        train_df = pd.read_csv(f"{base_path}/train.csv")
        test_df = pd.read_csv(f"{base_path}/test.csv")  # Ajout du test_df
        print(f"📊 Données chargées: {len(train_df)} exemples train, {len(test_df)} exemples test")

        # Échantillonnage pour l'évaluation
        sample_size = 50

        # Initialisation et évaluation avec ModelComparator (et non ModelEvaluator)
        comparator = ModelComparator(train_df, test_df)  # Utilisation du bon nom de classe

        # Génération des résultats avec les deux méthodes
        aggregated_scores = comparator.compare_models(sample_size)
        summary_comparison = comparator.compare_models_summaries(5)

        # Affichage des résultats
        display_comparison_results(aggregated_scores, summary_comparison)

        # Sauvegarde des résultats avec horodatage
        from datetime import datetime
        timestamp = datetime.now().strftime("%Y%m%d_%H%M")

        # Sauvegarde des deux types de résultats
        aggregated_scores.to_csv(f"{base_path}/aggregated_scores_{timestamp}.csv", index=False)
        summary_comparison.to_csv(f"{base_path}/summary_comparison_{timestamp}.csv", index=False)
        print(f"✅ Résultats sauvegardés avec timestamp: {timestamp}")

    except Exception as e:
        print(f"❌ Erreur: {e}")
        import traceback
        print(traceback.format_exc())

if __name__ == "__main__":
    main()




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📊 Données chargées: 12120 exemples train, 5195 exemples test
🖥️ Utilisation de: cuda


<ipython-input-10-dcbf92cbe81a>:122: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_df = self.train_df.groupby('language', group_keys=False).apply(
Comparaison des modèles: 45it [04:28,  5.97s/it]
<ipython-input-10-dcbf92cbe81a>:148: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_df = self.train_df.groupby('language').apply(
Génération des résumés: 75it [07:09,  5.72s/it]


📊 Scores ROUGE agrégés par modèle:

Moyenne globale:
          rouge1  rouge2  rougeL
model                           
gpt2       0.016  0.0062  0.0124
t5-base    0.088  0.0370  0.0785
t5-small   0.090  0.0387  0.0815

Moyenne par langue et modèle:
                     rouge1  rouge2  rougeL
language   model                           
Arabic     gpt2      0.0000  0.0000  0.0000
           t5-base   0.0000  0.0000  0.0000
           t5-small  0.0000  0.0000  0.0000
Bulgarian  gpt2      0.0000  0.0000  0.0000
           t5-base   0.0000  0.0000  0.0000
           t5-small  0.0000  0.0000  0.0000
Chinese    gpt2      0.0000  0.0000  0.0000
           t5-base   0.0000  0.0000  0.0000
           t5-small  0.0000  0.0000  0.0000
English    gpt2      0.1598  0.0670  0.1122
           t5-base   0.2476  0.1510  0.2476
           t5-small  0.2106  0.1402  0.2106
French     gpt2      0.0143  0.0050  0.0143
           t5-base   0.2491  0.1043  0.1693
           t5-small  0.2472  0.1047  0.1894
Ge

Performance Globale des Modèles
T5-Small (légèrement meilleur) :

- ROUGE-1 : 0.090 (9.0%)
- ROUGE-2 : 0.039 (3.9%)
- ROUGE-L : 0.082 (8.2%)

T5-Base (très proche) :
- ROUGE-1 : 0.088 (8.8%)
- ROUGE-2 : 0.037 (3.7%)
- ROUGE-L : 0.079 (7.9%)

GPT-2 (performances faibles) :
- ROUGE-1 : 0.016 (1.6%)
- ROUGE-2 : 0.006 (0.6%)
- ROUGE-L : 0.012 (1.2%)


Analyse par Langue
Meilleures performances :

Anglais :
- T5-Base : ROUGE-1 = 0.248 (24.8%)
- T5-Small : ROUGE-1 = 0.211 (21.1%)
- GPT-2 : ROUGE-1 = 0.160 (16.0%)

Français :
- T5-Base : ROUGE-1 = 0.249 (24.9%)
- T5-Small : ROUGE-1 = 0.247 (24.7%)

Espagnol :
- T5-Small : ROUGE-1 = 0.231 (23.1%)
- T5-Base : ROUGE-1 = 0.223 (22.3%)

Performances nulles (ROUGE = 0) :
Arabic, Bulgarian, Chinese, Greek, Hindi, Russian, Thai, Urdu
Points Clés 🔍

a) Biais Linguistique :
- Performances excellentes sur les langues européennes
- Échec total sur les langues non-latines
- Possible biais dans l'entraînement des modèles

b) Écarts-types :
- T5-Small : plus variable (σ = 0.129 pour ROUGE-1)
- T5-Base : légèrement plus stable (σ = 0.126 pour ROUGE-1)
- GPT-2 : moins variable mais performances faibles (σ = 0.067)

Améliorations Multilingues :
- Fine-tuning spécifique pour les langues non-latines
- Utilisation de tokenizers adaptés aux différentes langues
- Possible utilisation de modèles multilingues spécialisés (mT5, XLM-R)